In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)

# Physical parameters
m = 1.0    # Mass (kg)
L = 1.0    # Length (m)
b = 0.05   # Damping coefficient (kg/s)
g = 9.81   # Gravitational acceleration (m/s^2)
A = 1.0    # Amplitude of the forcing function
omega = 1.0 # Driving frequency
T = 20.0   # Total time (s)
N_cp = 500 # Number of collocation points

# Time discretization
t = torch.linspace(0, T, N_cp).unsqueeze(1)

# Initial conditions
theta_0 = 0.5  # Initial angular displacement (rad)
omega_0 = 0.0  # Initial angular velocity (rad/s)

# Forcing function
def forcing_function(t):
    return A * torch.sin(omega * t)

# PINN Vanilla Neural Network
class VanillaPINN(nn.Module):
    def __init__(self):
        super(VanillaPINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)
        )
    
    def forward(self, t):
        return self.net(t)

# Stacked PINN Neural Network
class StackedPINN(nn.Module):
    def __init__(self):
        super(StackedPINN, self).__init__()
        self.stacked_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(1, 100),
                nn.SiLU(),
                nn.Linear(100, 100),
                nn.SiLU(),
                nn.Linear(100, 2)
            ) for _ in range(4)
        ])
    
    def forward(self, t):
        out = self.stacked_layers[0](t)
        for layer in self.stacked_layers[1:]:
            out += layer(t)
        return out

# Physics-Informed Loss Function
def loss_function(t, theta_pred, omega_pred):
    # Compute the derivatives of theta and omega
    theta_t = torch.autograd.grad(theta_pred, t, grad_outputs=torch.ones_like(theta_pred), create_graph=True)[0]
    omega_t = torch.autograd.grad(omega_pred, t, grad_outputs=torch.ones_like(omega_pred), create_graph=True)[0]

    # Compute the second derivative of theta (angular acceleration)
    theta_tt = torch.autograd.grad(omega_t, t, grad_outputs=torch.ones_like(omega_t), create_graph=True)[0]

    # Forcing function
    F_t = forcing_function(t)

    # Equation of motion: d^2theta/dt^2 + (b/m) * dtheta/dt + (g/L) * sin(theta) = F(t)
    residual = theta_tt + (b / m) * omega_pred + (g / L) * torch.sin(theta_pred) - F_t

    # Mean squared error loss
    mse_residual = torch.mean(residual ** 2)

    return mse_residual

# Runge-Kutta (RK4) Integrator for Numerical Solution
class ForcedPendulumODE(nn.Module):
    def __init__(self, b, m, g, L):
        super(ForcedPendulumODE, self).__init__()
        self.b = b
        self.m = m
        self.g = g
        self.L = L
    
    def forward(self, x, t=None):
        theta = x[:, [0]]
        omega = x[:, [1]]
        dtheta_dt = omega
        domega_dt = -self.b / self.m * omega - self.g / self.L * torch.sin(theta) + forcing_function(t)
        return torch.cat([dtheta_dt, domega_dt], dim=-1)

# Numerical RK4 Solution
def runge_kutta_4th(ode, y0, t_eval):
    y_t = y0
    sol = [y_t]
    for t in t_eval[1:]:
        y_t = ode(y_t, t)
        sol.append(y_t)
    return torch.stack(sol).detach().numpy()

# Initialize models, optimizers, and ODE solver
vanilla_pinn = VanillaPINN()
stacked_pinn = StackedPINN()
pendulum_ode = ForcedPendulumODE(b, m, g, L)

# Numerical RK4 integration
y0 = torch.tensor([[theta_0, omega_0]])
t_eval = torch.linspace(0, T, N_cp)
sol_rk4 = runge_kutta_4th(pendulum_ode, y0, t_eval)

# Training the models
def train_model(model, epochs=10000, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_history = []
    
    t.requires_grad = True
    for epoch in range(epochs):
        # Forward pass: compute predicted theta and omega
        outputs = model(t)
        theta_pred = outputs[:, 0:1]
        omega_pred = outputs[:, 1:2]

        # Compute the physics-informed loss
        loss = loss_function(t, theta_pred, omega_pred)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Store the loss
        loss_history.append(loss.item())

        # Print progress every 1000 epochs
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    return model, loss_history

# Train vanilla and stacked models
vanilla_pinn, vanilla_loss = train_model(vanilla_pinn, epochs=5000)
stacked_pinn, stacked_loss = train_model(stacked_pinn, epochs=5000)

# Plot loss history
plt.figure(figsize=(8, 4))
plt.plot(vanilla_loss, label='Vanilla PINN Loss')
plt.plot(stacked_loss, label='Stacked PINN Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss History')
plt.grid(True)
plt.legend()
plt.show()

# Predict using trained models
theta_pred_vanilla, omega_pred_vanilla = vanilla_pinn(t).detach().numpy().T
theta_pred_stacked, omega_pred_stacked = stacked_pinn(t).detach().numpy().T

# Plot the results
plt.figure(figsize=(12, 6))

# Angular Displacement
plt.subplot(2, 1, 1)
plt.plot(t.detach().numpy(), theta_pred_vanilla, label='Vanilla PINN $\\theta(t)$')
plt.plot(t.detach().numpy(), theta_pred_stacked, label='Stacked PINN $\\theta(t)$')
plt.plot(t_eval.detach().numpy(), sol_rk4[:,0,0], label='Numerical RK4 $\\theta(t)$', linestyle='dashed')
plt.xlabel('Time [s]')
plt.ylabel('Angular Displacement [rad]')
plt.legend()
plt.grid(True)

# Angular Velocity
plt.subplot(2, 1, 2)
plt.plot(t.detach().numpy(), omega_pred_vanilla, label='Vanilla PINN $\\omega(t)$')
plt.plot(t.detach().numpy(), omega_pred_stacked, label='Stacked PINN $\\omega(t)$')
plt.plot(t_eval.detach().numpy(), sol_rk4[:,0,1], label='Numerical RK4 $\\omega(t)$', linestyle='dashed')
plt.xlabel('Time [s]')
plt.ylabel('Angular Velocity [rad/s]')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Verify that the stacked PINN parameters are small (e.g., any regularization or attention mechanisms)
for idx, param in enumerate(stacked_pinn.parameters()):
    print(f"Stacked PINN parameter {idx}: {param.data}")


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)

# Physical parameters
m = 1.0    # Mass (kg)
L = 1.0    # Length (m)
b = 0.05   # Damping coefficient (kg/s)
g = 9.81   # Gravitational acceleration (m/s^2)
A = 1.0    # Amplitude of the forcing function
omega = 1.0 # Driving frequency
T = 20.0   # Total time (s)
N_cp = 500 # Number of collocation points

# Time discretization
t = torch.linspace(0, T, N_cp).unsqueeze(1)

# Initial conditions
theta_0 = 0.5  # Initial angular displacement (rad)
omega_0 = 0.0  # Initial angular velocity (rad/s)

# Forcing function
def forcing_function(t):
    return A * torch.sin(omega * t)

# Enhanced PINN Neural Network with energy conservation
class EnhancedPINN(nn.Module):
    def __init__(self):
        super(EnhancedPINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 2)
        )
    
    def forward(self, t):
        return self.net(t)

# Energy function for physical constraint
def compute_total_energy(theta, omega):
    # Kinetic energy: (1/2) m L^2 omega^2
    kinetic_energy = 0.5 * m * L**2 * omega**2
    
    # Potential energy: m g L (1 - cos(theta))
    potential_energy = m * g * L * (1 - torch.cos(theta))
    
    # Total mechanical energy
    return kinetic_energy + potential_energy

# Physics-Informed Loss Function with energy conservation
def loss_function(t, theta_pred, omega_pred):
    # Compute the derivatives of theta and omega
    theta_t = torch.autograd.grad(theta_pred, t, grad_outputs=torch.ones_like(theta_pred), create_graph=True)[0]
    omega_t = torch.autograd.grad(omega_pred, t, grad_outputs=torch.ones_like(omega_pred), create_graph=True)[0]

    # Compute the second derivative of theta (angular acceleration)
    theta_tt = torch.autograd.grad(omega_t, t, grad_outputs=torch.ones_like(omega_t), create_graph=True)[0]

    # Forcing function
    F_t = forcing_function(t)

    # Equation of motion: d^2theta/dt^2 + (b/m) * dtheta/dt + (g/L) * sin(theta) = F(t)
    residual = theta_tt + (b / m) * omega_pred + (g / L) * torch.sin(theta_pred) - F_t

    # Mean squared error loss for the ODE residual
    mse_residual = torch.mean(residual ** 2)

    # Compute total energy at each point in time
    energy_pred = compute_total_energy(theta_pred, omega_pred)

    # Reference energy (initial energy)
    initial_energy = compute_total_energy(torch.tensor(theta_0), torch.tensor(omega_0))

    # Energy conservation loss: the energy should be close to the initial energy
    energy_loss = torch.mean((energy_pred - initial_energy) ** 2)

    # Total loss is a combination of the ODE residual and energy conservation loss
    return mse_residual + energy_loss

# Runge-Kutta (RK4) Integrator for Numerical Solution
class ForcedPendulumODE(nn.Module):
    def __init__(self, b, m, g, L):
        super(ForcedPendulumODE, self).__init__()
        self.b = b
        self.m = m
        self.g = g
        self.L = L
    
    def forward(self, x, t=None):
        theta = x[:, [0]]
        omega = x[:, [1]]
        dtheta_dt = omega
        domega_dt = -self.b / self.m * omega - self.g / self.L * torch.sin(theta) + forcing_function(t)
        return torch.cat([dtheta_dt, domega_dt], dim=-1)

# Numerical RK4 Solution
def runge_kutta_4th(ode, y0, t_eval):
    y_t = y0
    sol = [y_t]
    for t in t_eval[1:]:
        y_t = ode(y_t, t)
        sol.append(y_t)
    return torch.stack(sol).detach().numpy()

# Initialize models, optimizers, and ODE solver
enhanced_pinn = EnhancedPINN()
pendulum_ode = ForcedPendulumODE(b, m, g, L)

# Numerical RK4 integration
y0 = torch.tensor([[theta_0, omega_0]])
t_eval = torch.linspace(0, T, N_cp)
sol_rk4 = runge_kutta_4th(pendulum_ode, y0, t_eval)

# Training the model
def train_model(model, epochs=10000, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_history = []
    
    t.requires_grad = True
    for epoch in range(epochs):
        # Forward pass: compute predicted theta and omega
        outputs = model(t)
        theta_pred = outputs[:, 0:1]
        omega_pred = outputs[:, 1:2]

        # Compute the physics-informed loss
        loss = loss_function(t, theta_pred, omega_pred)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Store the loss
        loss_history.append(loss.item())

        # Print progress every 1000 epochs
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

    return model, loss_history

# Train the enhanced model
enhanced_pinn, enhanced_loss = train_model(enhanced_pinn, epochs=5000)

# Plot loss history
plt.figure(figsize=(8, 4))
plt.plot(enhanced_loss, label='Enhanced PINN Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss History (Enhanced PINN with Energy Conservation)')
plt.grid(True)
plt.legend()
plt.show()

# Predict using trained models
theta_pred, omega_pred = enhanced_pinn(t).detach().numpy().T

# Plot the results
plt.figure(figsize=(12, 6))

# Angular Displacement
plt.subplot(2, 1, 1)
plt.plot(t.detach().numpy(), theta_pred, label='Enhanced PINN $\\theta(t)$')
plt.plot(t_eval.detach().numpy(), sol_rk4[:,0,0], label='Numerical RK4 $\\theta(t)$', linestyle='dashed')
plt.xlabel('Time [s]')
plt.ylabel('Angular Displacement [rad]')
plt.legend()
plt.grid(True)

# Angular Velocity
plt.subplot(2, 1, 2)
plt.plot(t.detach().numpy(), omega_pred, label='Enhanced PINN $\\omega(t)$')
plt.plot(t_eval.detach().numpy(), sol_rk4[:,0,1], label='Numerical RK4 $\\omega(t)$', linestyle='dashed')
plt.xlabel('Time [s]')
plt.ylabel('Angular Velocity [rad/s]')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Verify learned parameters
for idx, param in enumerate(enhanced_pinn.parameters()):
    print(f"Enhanced PINN parameter {idx}: {param.data}")
